<a href="https://colab.research.google.com/github/danhively/CS4499/blob/main/Mid_Project_Hively.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mid-Project: Predicting Sports Winners
Data Science and Applied Machine Learning \\
Adpopted from material from Dr. Kerby \\
Student Name: Dan Hively \\
Date Due: April 5, 2023, 11:59 PM

### Question 1: Gather Data from 2021-2022
*Gather* data from [Basketball Reference](https://www.basketball-reference.com/leagues/NBA_2022.html) <br>

Data can be downloaded by clicking on the `Share and Export` drop-down list at the middle top of available tables. Start off by clicking `Schedule and Results` and downloading the monthly games tables. Combine these into one table. Then import the full games-results table into python. 


In [209]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Gather data - This will be the baseline data set for NBA_2021-22.
dfNBA202122 = pd.read_csv('https://raw.github.com/danhively/CS4499/main/Mid-Poject/NBA_2021-22.csv',index_col=0)

# This will be commented out before submission.
# Mount my Google Drive
# I want to save a CSV after manipulations to check it.
#from google.colab import drive
#mntPath = '/content/drive/'
#drive.mount(mntPath, force_remount=True)
#svPath = mntPath + "My Drive/CS4499/Mid-Project/"
#dfNBA202122.to_csv(svPath + "dfNBA202122.csv")

#######################################
#   Useful Functions                  #
#######################################

# Function to rename a column by position
def rename_column_by_position(df, position, new_name):
    if 0 <= position < len(df.columns):
        columns = df.columns.tolist()
        columns[position] = new_name
        df.columns = columns
    else:
        raise ValueError(f"Invalid position. DataFrame has {len(df.columns)} columns.")

# Function to compare PTS1 and PTS
def compare_scores(row):
    if row['PTS.1'] > row['PTS']:
        return True
    else:
        return False

# Drop specified columns from a DataFrame.
def drop_columns(df, columns_to_drop):
    return df.drop(columns=columns_to_drop)        


### Question 2: EDA and Preprocessing
Explore your dataset and make any preprocessing adjustments necessary. You must perform at least 3 operations on exploring your data. Each one must provide unqiue insight to the data. At the bottom of this problem, write a paragraph (at least 5 sentences) explaining what you did and found out. 

In [210]:
# Print some basic information about the dataset.
# Print the shape of the DataFrame
print('----------------------------')
print('Shape')
print('----------------------------')
print(dfNBA202122.shape)

# Print some summary statistics of the DataFrame
print('----------------------------')
print('statistics')
print('----------------------------')
print(dfNBA202122.describe())

# Print the list of columns in the CSV file.
print('----------------------------')
print('Columns')
print('----------------------------')
print(dfNBA202122.columns)

# Print the first five rows of the DataFrame
print('----------------------------')
print('First 5 rows')
print('----------------------------')
print(dfNBA202122.head())

# Count the number of NaN values in each column
print('----------------------------')
print('Number of NaN values')
print('----------------------------')
nan_count = dfNBA202122.isna().sum()
print(nan_count)

# Find columns with NaN values
null_columns = dfNBA202122.columns[dfNBA202122.isna().any()].tolist()

# Display column headings with null values
print('----------------------------')
print('Column headings with null values')
print('----------------------------')
print(null_columns)

----------------------------
Shape
----------------------------
(1171, 10)
----------------------------
statistics
----------------------------
               PTS        PTS.1  Notes
count  1171.000000  1171.000000    0.0
mean    111.382579   112.558497    NaN
std      12.563364    12.510739    NaN
min      75.000000    73.000000    NaN
25%     103.000000   104.000000    NaN
50%     111.000000   112.000000    NaN
75%     120.000000   121.000000    NaN
max     154.000000   154.000000    NaN
----------------------------
Columns
----------------------------
Index(['Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1',
       'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes'],
      dtype='object')
----------------------------
First 5 rows
----------------------------
                  Start (ET)        Visitor/Neutral  PTS         Home/Neutral  \
Date                                                                            
Tue, Dec 22, 2020      7:00p  Golden State Warr

What I discoverwed about the data.


The data

### Question 3: Feature Engineering

Create a column for whether or not the home team won the game; call it `HomeWin`. Use Pandas and your knowledge of Python to fill in this data. This will become our target -- ie what we are trying to predict. 

In [211]:
# Create a copy of the dfNBA202122 DataFrame
winsdf = dfNBA202122.copy()

# We have 2 columns that have the same name. Let's rename them.
# Rename the columns
#rename_column_by_position(winsdf, 3, 'V_Score')
#rename_column_by_position(winsdf, 5, 'H_Score')

# Create a new column 'HomeWin' based on whether 'PTS.1' is greater than 'PTS' .
# True if PTS.1 is greater than PTS otherwise False.
winsdf['HomeWin'] = winsdf.apply(compare_scores, axis=1)

# Display the new DataFrame with the added column.
print("\nNew DataFrame with 'HomeWin' column (winsdf):")
print(winsdf.columns.tolist())
print(winsdf.head())
print(winsdf['HomeWin'])


New DataFrame with 'HomeWin' column (winsdf):
['Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1', 'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes', 'HomeWin']
                  Start (ET)        Visitor/Neutral  PTS         Home/Neutral  \
Date                                                                            
Tue, Dec 22, 2020      7:00p  Golden State Warriors   99        Brooklyn Nets   
Tue, Dec 22, 2020     10:00p   Los Angeles Clippers  116   Los Angeles Lakers   
Wed, Dec 23, 2020      7:00p      Charlotte Hornets  114  Cleveland Cavaliers   
Wed, Dec 23, 2020      7:00p        New York Knicks  107       Indiana Pacers   
Wed, Dec 23, 2020      7:00p             Miami Heat  107        Orlando Magic   

                   PTS.1 Unnamed: 6 Unnamed: 7 Attend.  \
Date                                                     
Tue, Dec 22, 2020    125  Box Score        NaN       0   
Tue, Dec 22, 2020    109  Box Score        NaN       0   
Wed, Dec 23, 2020  

### Question 4: Feature Engineering II
Create two columns for how many games the home and visitor teams have won thus far in the season. Call them `HomeNumWins` and `VisitorNumWins`.
 
For example Game 1 for the Utah Jazz was against the OKC Thunder. The Jazz were the home team, OKC was the road team. Since this is game 1 for both teams `HomeNumWins` and `VisitorNumWins` will be 0. The Jazz won this game and were the road team for their next game (against the Sacramento Kings),  `VisitorNumWins` should be a 1 for this value. OKC would go on to lose their second game (on the road to Houston), Game 3 was their first home game. So for OKC's 3rd game (against the 76ers) `HomeNumWins` would be 0. 

Hint: Store each team and the number of intial wins (0) in a Python dictionary. You can then increment this value as you go through the dataset and use this information to update your feature. 



In [212]:
# Create two new columns 'HomeNumWins' and 'VisitorNumWins'
winsdf['HomeNumWins'] = 0
winsdf['VisitorNumWins'] = 0

# Dictionary to store wins for each team
team_wins = {}

# Get all team names and print them.
teams = set(winsdf['Home/Neutral']).union(winsdf['Visitor/Neutral'])
print(teams)


# Initialize the dictionary with 0 wins for each team
for team in teams:
    team_wins[team] = 0

for index, row in winsdf.iterrows():
    home_team = row['Home/Neutral']
    visitor_team = row['Visitor/Neutral']
    home_win = row['HomeWin']
    
    if home_win:
        team_wins[home_team] += 1
        winsdf.at[index, 'HomeNumWins'] = team_wins[home_team]
        winsdf.at[index, 'VisitorNumWins'] = team_wins[visitor_team]
    else:
        team_wins[visitor_team] += 1
        winsdf.at[index, 'HomeNumWins'] = team_wins[home_team]
        winsdf.at[index, 'VisitorNumWins'] = team_wins[visitor_team]

print(winsdf.head())
print(winsdf.columns.tolist())

# Save the resulting DataFrame to a CSV file just to check it has been
# modified properly. This will be commented out before submission.
#print('===========================================')
#winsdf.to_csv(svPath + "winsdf.csv")
#print("CSV file saved as 'winsdf.csv'")

{'Chicago Bulls', 'Milwaukee Bucks', 'Charlotte Hornets', 'Sacramento Kings', 'San Antonio Spurs', 'Orlando Magic', 'Denver Nuggets', 'Detroit Pistons', 'Los Angeles Clippers', 'New Orleans Pelicans', 'Houston Rockets', 'Boston Celtics', 'Philadelphia 76ers', 'Oklahoma City Thunder', 'Cleveland Cavaliers', 'Atlanta Hawks', 'Toronto Raptors', 'Brooklyn Nets', 'Memphis Grizzlies', 'Golden State Warriors', 'Portland Trail Blazers', 'Utah Jazz', 'Washington Wizards', 'Los Angeles Lakers', 'Phoenix Suns', 'Dallas Mavericks', 'Miami Heat', 'New York Knicks', 'Indiana Pacers', 'Minnesota Timberwolves'}
                  Start (ET)        Visitor/Neutral  PTS         Home/Neutral  \
Date                                                                            
Tue, Dec 22, 2020      7:00p  Golden State Warriors   99        Brooklyn Nets   
Tue, Dec 22, 2020     10:00p   Los Angeles Clippers  116   Los Angeles Lakers   
Wed, Dec 23, 2020      7:00p      Charlotte Hornets  114  Cleveland Caval

In [213]:
# Example with old, 2018-2019, season data
import pandas as pd
import numpy as np

sports = pd.read_csv("https://gitlab.com/CEADS/DrKerby/python/-/raw/master/basketball.csv")
sports

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Tue Oct 16 2018,8:00p,Philadelphia 76ers,87,Boston Celtics,105,Box Score,NaN,18624,NaN
1,Tue Oct 16 2018,10:30p,Oklahoma City Thunder,100,Golden State Warriors,108,Box Score,NaN,19596,NaN
2,Wed Oct 17 2018,7:00p,Milwaukee Bucks,113,Charlotte Hornets,112,Box Score,NaN,17889,NaN
3,Wed Oct 17 2018,7:00p,Brooklyn Nets,100,Detroit Pistons,103,Box Score,NaN,20332,NaN
4,Wed Oct 17 2018,8:00p,New Orleans Pelicans,131,Houston Rockets,112,Box Score,NaN,18055,NaN
...,...,...,...,...,...,...,...,...,...,...
1307,Sun Jun 2 2019,8:00p,Golden State Warriors,109,Toronto Raptors,104,Box Score,NaN,20014,NaN
1308,Wed Jun 5 2019,9:00p,Toronto Raptors,123,Golden State Warriors,109,Box Score,NaN,19596,NaN
1309,Fri Jun 7 2019,9:00p,Toronto Raptors,105,Golden State Warriors,92,Box Score,NaN,19596,NaN
1310,Mon Jun 10 2019,9:00p,Golden State Warriors,106,Toronto Raptors,105,Box Score,NaN,20144,NaN


In [214]:
sports['Unnamed: 6'].unique()

array(['Box Score'], dtype=object)

In [215]:
sports['Unnamed: 7'].unique()

array([nan, 'OT', '2OT', '3OT', '4OT'], dtype=object)

In [216]:
sports['Notes'].unique()

array([nan, 'at Mexico City Mexico', 'at London England'], dtype=object)

In [217]:
from collections import defaultdict
won_last = defaultdict(int)

won_last
# Create HomeWin column
sports['HomeWin'] = sports['PTS.1'] > sports['PTS']
sports.head()
# Create new columns and initialize with zero
sports['HomeLastWin'] = 0
sports['VisitorLastWin'] = 0

sports.head()

# Take a slice first, sports[:5]
for index, row in sports.iterrows():
  #print(index, row['Home/Neutral'], row['PTS.1'])
  home_team = row['Home/Neutral']
  visitor_team = row['Visitor/Neutral']
  sports.loc[index,'HomeLastWin'] = won_last[home_team]
  sports.loc[index,'VisitorLastWin'] = won_last[visitor_team]

  # Who won this game, to record it in the next game?
  #print(int(row['HomeWin']))  # typecast boolean to int (false->0, true->1)
  won_last[home_team] = int(row['HomeWin'])
  won_last[visitor_team] = 1 - int(row['HomeWin'])

sports.head(50)
#type(sports['Date'][0])

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,HomeWin,HomeLastWin,VisitorLastWin
0,Tue Oct 16 2018,8:00p,Philadelphia 76ers,87,Boston Celtics,105,Box Score,NaN,18624,NaN,True,0,0
1,Tue Oct 16 2018,10:30p,Oklahoma City Thunder,100,Golden State Warriors,108,Box Score,NaN,19596,NaN,True,0,0
2,Wed Oct 17 2018,7:00p,Milwaukee Bucks,113,Charlotte Hornets,112,Box Score,NaN,17889,NaN,False,0,0
3,Wed Oct 17 2018,7:00p,Brooklyn Nets,100,Detroit Pistons,103,Box Score,NaN,20332,NaN,True,0,0
4,Wed Oct 17 2018,8:00p,New Orleans Pelicans,131,Houston Rockets,112,Box Score,NaN,18055,NaN,False,0,0
5,Wed Oct 17 2018,7:00p,Memphis Grizzlies,83,Indiana Pacers,111,Box Score,NaN,17923,NaN,True,0,0
6,Wed Oct 17 2018,10:30p,Denver Nuggets,107,Los Angeles Clippers,98,Box Score,NaN,19068,NaN,False,0,0
7,Wed Oct 17 2018,7:30p,Atlanta Hawks,107,New York Knicks,126,Box Score,NaN,18249,NaN,True,0,0
8,Wed Oct 17 2018,7:00p,Miami Heat,101,Orlando Magic,104,Box Score,NaN,19191,NaN,True,0,0
9,Wed Oct 17 2018,10:30p,Dallas Mavericks,100,Phoenix Suns,121,Box Score,NaN,18055,NaN,True,0,0


### Question 5: Model

Train a decision tree model to determine if the home team won (a 1 is a win and a 0 is a loss. Evaluate your model. At the bottom of this problem write 2 paragraphs explaining your models, its performance and any possible issues you see with it. Comment on the most "important" features in the model.

Remember to split the dataset into training and testing sets and delete any columns that you deem unuseful. (Hint: Do not use any information that you wouldn't know before the game is played, the idea is to create a model that predicts wins)



In [218]:
dfNBA202122 = pd.read_csv('https://raw.github.com/danhively/CS4499/main/Mid-Poject/NBA_2021-22.csv')

# Duplicate the dataframe with only columns we need for the model.
newdf = dfNBA202122[['Visitor/Neutral', 'Home/Neutral', 'PTS', 'PTS.1']].copy()

# Create 'HomeWin' column
newdf.loc[:, 'HomeWin'] = (newdf['PTS'] < newdf['PTS.1']).astype(int)
print(newdf)

# Prepare the data for training and testing.
X = newdf.drop(columns='HomeWin')
X = pd.get_dummies(X)  
y = newdf['HomeWin']

print(X)
print(y)

# Split the data into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.7, random_state=35)

# Train the decision tree model
clf = DecisionTreeClassifier(random_state=42)
clf.fit(Xtrain, ytrain)

# Predict the outcomes for the test set
y_pred = clf.predict(Xtest)

# Evaluate the model
print("Accuracy:", accuracy_score(ytest, y_pred))
print("Confusion Matrix:\n", confusion_matrix(ytest, y_pred))
print("Classification Report:\n", classification_report(ytest, y_pred))


            Visitor/Neutral         Home/Neutral  PTS  PTS.1  HomeWin
0     Golden State Warriors        Brooklyn Nets   99    125        1
1      Los Angeles Clippers   Los Angeles Lakers  116    109        0
2         Charlotte Hornets  Cleveland Cavaliers  114    121        1
3           New York Knicks       Indiana Pacers  107    121        1
4                Miami Heat        Orlando Magic  107    113        1
...                     ...                  ...  ...    ...      ...
1166        Milwaukee Bucks         Phoenix Suns  108    118        1
1167           Phoenix Suns      Milwaukee Bucks  100    120        1
1168           Phoenix Suns      Milwaukee Bucks  103    109        1
1169        Milwaukee Bucks         Phoenix Suns  123    119        0
1170           Phoenix Suns      Milwaukee Bucks   98    105        1

[1171 rows x 5 columns]
      PTS  PTS.1  Visitor/Neutral_Atlanta Hawks  \
0      99    125                              0   
1     116    109                 

### Question 6: Add data from the previous season
Go back to the website and download the team standings from the PREVIOUS season. You will find it in the `Standings` tab. Keep just the number of wins and create two new columns called `HomeWins2020` and `VisitorWins2020`.

In [ ]:
# Gather data - This will be the baseline data set for NBA_2020-21.
dfNBA202021 = pd.read_csv('https://raw.github.com/danhively/CS4499/main/Mid-Poject/NBA_2020-21.csv')



# Combine the dataframes.
frames = [dfNBA202122, dfNBA202021]
resultdf = pd.concat(frames)
print(resultdf)

### Question 7: Repeat Model

Repeat your analysis from Question 5.

### Question 8: Analysis and Discussion
What was your best-performing model? \\
How well did it do? \\
Was it overfit? \\
What seems to be the most important feature(s)? \\
How well did your model perform compared to blindly guessing the Home Team won?